# More Complex Graph Types
In addition to simple linear graphs, Akka.Streams also allows for more complex graphs - including the merging of `Source<T>` / `Sink<T>` stages and splitting output from `Flow<T>`.

## Fan-In Stages
Stages that combine multiple `Source<T>`s together are called ["fan-in" stages](https://getakka.net/articles/streams/builtinstages.html#fan-in-stages) and they can be used to combine multiple data sources together into contiguous streams.

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"
#r "nuget: Akka.Streams.IAsyncEnumerable, 0.1.0"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample");

IMaterializer materializer = actorSystem.Materializer();

// a source representing a range of integers
Source<int, NotUsed> source1 = Source.From(Enumerable.Range(1, 10));

// a source representing a single string value
Source<string, NotUsed> source2 = Source.Single("a");

// let's combine these two sources such that we create 10 int / string tuples
IAsyncEnumerable<(int i, string s)> merged1 = source1.Zip(source2).RunAsAsyncEnumerable(materializer);

await foreach(var (i, s) in merged1){
    Console.WriteLine($"{i}-->{s}");
}


Installed Packages Akka.Streams, 1.4.24 Akka.Streams.IAsyncEnumerable, 0.1.0

1-->a


You'll notice that we only produced a single output with code sample above:

```
1-->a
```

That's because our `Source.Single` can only produce a single element before it's completed - if we change this graph to use a `Source.Repeat` instead, we can fully populate this list.

In [ ]:
// REPEAT this value each time we're pulled
Source<string, NotUsed> source3 = Source.Repeat("a");

// let's combine these two sources such that we create 10 int / string tuples
IAsyncEnumerable<(int i, string s)> merged1 = source1.Zip(source3).RunAsAsyncEnumerable(materializer);

await foreach(var (i, s) in merged1){
    Console.WriteLine($"{i}-->{s}");
}

1-->a
2-->a
3-->a
4-->a
5-->a
6-->a
7-->a
8-->a
9-->a
10-->a


## Fan-Out Stages
In addition to fanning-in data from multiple sources into a single stream, it's also possible to pipe output from one `Source<T>` to multiple substreams using a fanout stage!

In [ ]:
// create a source of 1-10
Source<int, NotUsed> sharedSource = Source.From(Enumerable.Range(0,10));

// create a substream that will print out even / odd status
var (completeStatus, statusSink) = Sink.Seq<string>().PreMaterialize(materializer);

Sink<int, NotUsed> numFlow = Flow.Create<int>().Select(x => x % 2 == 0 ? $"{x} is even" : $"{x} is odd").To(statusSink);

// create a substream that will compute sum of all numbers
var (completeSum, sumSink) = Sink.Sum<int>((i, n) => i + n).PreMaterialize(materializer);

var combined = Sink.Combine<int>(i => new Broadcast<int>(2), statusSink, sumSink);

//sharedSource.To(numFlow).

Error: (12,21): error CS0305: Using the generic method 'Sink.Combine<TIn, TOut, TMat>(Func<int, IGraph<UniformFanOutShape<TIn, TOut>, TMat>>, Sink<TOut, NotUsed>, Sink<TOut, NotUsed>, params Sink<TOut, NotUsed>[])' requires 3 type arguments